# Allen Majewski 20200107

Notes:

This is a git repository with the following structure:

`README.md`

`src/`

`-- taxitip_model.ipynb` (this file)

`data/`

   `-- 2015_<MM>_100k.csv` 
   
   `-- 2015_weather.csv`
   
   `-- holidays.csv`
   
The data files are in the gitignore so they are not pushed.

The discussion will follow inline with the code.

Below is the problem statement.



# dotdata ML task

### Data
	1. NYC taxi trip history data from 2015/01 – 2015/12
	2. Daily weather data from 2015/01 – 2015/12
	3. Public holiday data for the year of 2015

## Problem
Build an ML model to predict whether each trip has over 20% tip rate or not.

### Expected outputs

	1. Please submit the code you developed to build predictive model(s) as Jupyter notebook(s) on Python3 kernel.
	2. Please summarize the key conclusion of your analysis. This report should include the following:
		a. The most significant features that affect each trip’s tip percentage.
		b. The performance of your predictive model(s), and suggest what additional dataset you’d like to include to improve the performance of your models.

### Note
	1.  Please assume the client wants to get the model with high prediction accuracy.
	2.  Please assume the code would be reviewed by your team members, and be further developed.

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import random
# import mysql.connector
# import json
# import math
# import datetime
# import dateutil
# import argparse
import matplotlib.pyplot as plt
# from operator import add, truediv
from sklearn import preprocessing
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression, Lasso, LassoCV, LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVR, LinearSVC
from sklearn.naive_bayes import GaussianNB
# from scipy.stats.stats import pearsonr
import calendar
import datetime

#################################
#  some functions I use later   #
#################################

def get_min_max(seq):
    '''
    simple min max scaler
    '''
    min_ = min(seq)
    max_ = max(seq)
    seq = (seq-min_)/(max_ - min_)
    return [(min_,max_),seq]

def lmap(func, alist):
    '''
    returns a list instead of a map object
    '''
    return list(map(func, alist))

def anomaly(seq):
    '''
    statistical anomaly
    '''
    return (seq - np.average(seq))/np.std(seq)



os.chdir('../data')

# load up fare data

`bigdf` is all the months

In [2]:
dfs = []

for infile in sorted([ _ for _ in os.listdir('.') if _.endswith('100k.csv')]):
    sys.stdout.write('\r'+f'loading {infile}')
    df = pd.read_csv(infile)
    dfs.append(df)
    
bigdf = pd.concat(dfs, ignore_index=True)

bigdf.tail(5)

loading 2015-12_100k.csv

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood
1199995,2,2015-12-16 19:00:26,2015-12-16 19:08:39,2,1.25,-73.974289,40.779854,1,N,-73.975204,...,1.4,0.00,0.3,10.70,10023,Manhattan,Upper West Side,10025,Manhattan,Upper West Side
1199996,1,2015-12-07 06:41:33,2015-12-07 06:45:03,1,0.70,-73.991707,40.749870,1,N,-73.980362,...,0.0,0.00,0.3,5.80,10001,Manhattan,Chelsea and Clinton,10016,Manhattan,Gramercy Park and Murray Hill
1199997,1,2015-12-02 09:18:35,2015-12-02 09:30:59,1,1.40,-73.955223,40.773376,1,N,-73.968201,...,0.0,0.00,0.3,10.30,10028,Manhattan,Upper East Side,10022,Manhattan,Gramercy Park and Murray Hill
1199998,1,2015-12-29 09:02:47,2015-12-29 09:08:49,1,0.60,-73.990509,40.742191,1,N,-73.979141,...,0.0,0.00,0.3,6.30,10010,Manhattan,Gramercy Park and Murray Hill,10010,Manhattan,Gramercy Park and Murray Hill
1199999,2,2015-12-09 21:57:56,2015-12-09 22:18:33,2,10.00,-73.870743,40.773689,1,N,-73.860840,...,0.0,5.54,0.3,36.34,11369,Queens,West Queens,10462,Bronx,Southeast Bronx




## Defining the objective: 20% of what?

We will consider whether the tip is 20% of the fare *without taxes*, as discussed with Michal over email.  Noting that $\text{total_amount}$ includes tax, tip, and all charges, let's define 

$\text{user_fare} = \text{total_amount} - \text{tip_amount} - \text{mta_tax}$

Then tip fraction is just

$\text{tip_fraction} = \text{tip_amount}/\text{user_fare}$

In [3]:
USER_FARE    = np.array(bigdf.total_amount) - np.array(bigdf.tip_amount) - np.array(bigdf.mta_tax)
TIP_FRACTION = np.array(bigdf.tip_amount)/USER_FARE

bigdf['user_fare']    = USER_FARE
bigdf['tip_fraction'] = TIP_FRACTION


## We will treat tip prediction as both a regression problem and a classification problem

We will use regression here to predict the tip_fraction and/or tip, but the problem is centrally to *classify* whether the tip is above a threshold (20%).  

Let us then implement this boolean to be predicted by a *classifier*:




`initialize:` 

$\text{tip_over_twenty_percent} \leftarrow 0$.


`if:`

$\text{tip_fraction} \ge 0.2$

`then:`

$\text{tip_over_twenty_percent} \leftarrow 1$



In [4]:
TIP_OVER_TWENTY = [0]*len(bigdf)

for idx, tip_fraction in enumerate(bigdf.tip_fraction.values):
    if tip_fraction >= 0.2:
        TIP_OVER_TWENTY[idx] = 1

bigdf['tip_over_twenty_percent'] = TIP_OVER_TWENTY

bigdf.head(5)

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,total_amount,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,user_fare,tip_fraction,tip_over_twenty_percent
0,2,2015-01-06 11:39:29,2015-01-06 11:49:15,1,1.78,-73.999619,40.743599,1,N,-73.992203,...,9.80,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,9.3,0.000000,0
1,1,2015-01-13 09:18:29,2015-01-13 09:23:40,1,2.10,-73.981956,40.778290,1,N,-73.962173,...,10.75,10023,Manhattan,Upper West Side,10024,Manhattan,Upper West Side,7.8,0.314103,1
2,2,2015-01-16 07:15:44,2015-01-16 07:26:42,1,2.33,-73.991188,40.742226,1,N,-73.981613,...,11.80,10010,Manhattan,Gramercy Park and Murray Hill,10019,Manhattan,Chelsea and Clinton,10.3,0.097087,0
3,1,2015-01-23 11:56:05,2015-01-23 12:13:20,1,2.10,-73.959297,40.763336,1,N,-73.979996,...,13.30,10065,Manhattan,Upper East Side,10023,Manhattan,Upper West Side,12.8,0.000000,0
4,1,2015-01-24 10:11:48,2015-01-24 10:18:32,2,0.90,-73.971832,40.764751,1,N,-73.984047,...,7.30,10065,Manhattan,Upper East Side,10019,Manhattan,Chelsea and Clinton,6.8,0.000000,0


A small test of the above manipulations

In [5]:
TEST_SIZE=5000

for i in range(TEST_SIZE):
    
    row_idx = random.randint(0,len(bigdf))
    sys.stdout.write(f'\rtesting row {row_idx}')

    row                     = bigdf.iloc[row_idx]
    user_fare               = row.user_fare
    tip_amount              = row.tip_amount
    tip_fraction            = row.tip_fraction
    tip_over_twenty_percent = row.tip_over_twenty_percent

    assert(tip_fraction == tip_amount/user_fare)
    
    if tip_fraction < 0.2:
        assert(tip_over_twenty_percent == 0)
    else:
        assert(tip_over_twenty_percent == 1)

testing row 143456

## Use of auxiliary data: holidays, weather

First manipulate the weather and holiday dataframes themselves.  Plan to join with principle data on 'pickup_date'.



### examine weather data

In [6]:
weather = pd.read_csv('2015_weather.csv', sep=';')
weather.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow
0,01.01.15,1,NaN,NaN,NaN
1,02.01.15,4,NaN,NaN,NaN
2,03.01.15,3,1.0,NaN,1.0
3,04.01.15,9,1.0,NaN,NaN
4,05.01.15,2,NaN,NaN,NaN




#### plan to consider weather data:
* impute 0 where NaN is in the Rain, Fog, Snow columns and use directly as feature since it is bool
* consider minmax-scaled temperature in deg C, and also temperature anomaly
* fix weird date format


In [7]:
# impute 0 for NaN
for column in ['Rain', 'Fog', 'Snow']:
    weather[column] = lmap(np.nan_to_num, weather[column].values)

# minmax scale temp    
weather['avg_temp_C_scaled'] = get_min_max(weather.avg_temp_C.values)[1]

# anomaly
weather['avg_temp_C_anomaly'] = anomaly(weather.avg_temp_C.values)

# fix funny date
def fixdate(date):
    dd   = date.split('.')[0]
    mm   = date.split('.')[1]
    yyyy = '20'+date.split('.')[2]
    
    return f'{yyyy}-{mm}-{dd}'

weather['pickup_date'] = lmap(fixdate, weather.pickup_date.values)

weather.head()

,pickup_date,avg_temp_C,Rain,Fog,Snow,avg_temp_C_scaled,avg_temp_C_anomaly
0,2015-01-01,1,0.0,0.0,0.0,0.302326,-1.226826
1,2015-01-02,4,0.0,0.0,0.0,0.372093,-0.942334
2,2015-01-03,3,1.0,0.0,1.0,0.348837,-1.037165
3,2015-01-04,9,1.0,0.0,0.0,0.488372,-0.468179
4,2015-01-05,2,0.0,0.0,0.0,0.325581,-1.131996



#### exame holiday data

In [8]:
holidays = pd.read_csv('holidays.csv', sep=';')
holidays.head(10)

,Date,Holiday
0,01.01.15,New Years Day
1,19.01.15,Martin Luther King Jr. Day
2,12.02.15,Lincoln's Birthday
3,16.02.15,Presidents' Day
4,10.05.15,Mother's Day
5,25.05.15,Memorial Day
6,21.06.15,Father's Day
7,03.07.15,Independence Day (observed)
8,07.09.15,Labor Day
9,12.10.15,Columbus Day


### plan to consider holidays/dates:

1) fix funny date format in this df

2) just add a boolean  

`is_holiday` 

  to the principle data 


3) I have a feeling the day of the week matters (perhaps fridays have more generous passengers than mondays)

`weekday` 

as in monday-sunday

`M T W R F Sa Su` 

to be one-hot encoded and a boolean 

`is_weekend`.




In [9]:
holidays['datetime'] = lmap(fixdate, holidays.Date.values)
holidays['pickup_date'] = [ _.split(' ')[0] for _ in holidays.datetime.values ]
holidays = holidays.drop(columns=['Date', 'datetime'])
holidays

,Holiday,pickup_date
0,New Years Day,2015-01-01
1,Martin Luther King Jr. Day,2015-01-19
2,Lincoln's Birthday,2015-02-12
3,Presidents' Day,2015-02-16
4,Mother's Day,2015-05-10
5,Memorial Day,2015-05-25
6,Father's Day,2015-06-21
7,Independence Day (observed),2015-07-03
8,Labor Day,2015-09-07
9,Columbus Day,2015-10-12


#### add is_holiday to bigdf

In [10]:
# add is_holiday to bigdf
#
is_holiday_dates = holidays.pickup_date.values
#print(is_holiday_dates)

bigdf['pickup_date'] = [ _.split(' ')[0] for _ in bigdf.pickup_datetime.values ]
bigdf['is_holiday'] = [ 1 if day in is_holiday_dates else 0 for day in list(bigdf['pickup_date']) ]


#### join bigdf, weather, holiday on 'pickup_date'

In [11]:
bigdf = bigdf.merge(holidays, how='outer', on='pickup_date')
bigdf = bigdf.merge(weather, how='outer', on='pickup_date')

# lets look at christmas day
bigdf[bigdf.pickup_date=='2015-12-25']

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,tip_over_twenty_percent,pickup_date,is_holiday,Holiday,avg_temp_C,Rain,Fog,Snow,avg_temp_C_scaled,avg_temp_C_anomaly
1198349,2,2015-12-25 16:27:44,2015-12-25 16:35:56,1,2.50,-73.987289,40.770222,1,N,-73.957420,...,0,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1198350,1,2015-12-25 12:24:49,2015-12-25 12:26:46,1,0.80,-73.947639,40.787636,1,N,-73.939316,...,0,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1198351,1,2015-12-25 19:45:29,2015-12-25 19:49:46,1,1.00,-73.933418,40.763176,1,N,-73.945854,...,1,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1198352,1,2015-12-25 17:09:49,2015-12-25 17:16:09,1,1.90,-73.974541,40.757076,1,N,-73.989700,...,0,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1198353,1,2015-12-25 16:58:52,2015-12-25 17:13:10,2,1.80,-73.984985,40.748184,1,N,-73.982368,...,0,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,2,2015-12-25 23:10:23,2015-12-25 23:18:09,6,1.90,-73.979012,40.785275,1,N,-73.982971,...,1,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1199996,2,2015-12-25 09:48:10,2015-12-25 09:56:42,1,1.81,-73.955025,40.769787,1,N,-73.978012,...,1,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1199997,1,2015-12-25 14:39:38,2015-12-25 15:01:38,1,9.50,-73.873070,40.774132,1,N,-73.979591,...,1,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468
1199998,2,2015-12-25 16:05:09,2015-12-25 16:09:12,1,1.10,-73.988861,40.773930,1,N,-73.999367,...,1,2015-12-25,1,Christmas Day,17,1.0,0.0,0.0,0.674419,0.290468


#### add weekday (monday-friday) and is_weekend

* perhaps folks are grumpy on monday and generous on friday

* weekend != workdays, maybe family outings arent the same as business travel

* let us keep also the hour from the datetime.  (Maybe night travelers are drunk and generous, etc).

In [12]:
def pickup_date_to_weekday(input_string):
    '''
    > pickup_date_to_weekday('2015-12-26')
    'Saturday'
    '''
    inp = input_string.strip()
    yyyy = int(inp.split('-')[0])
    mm   = int(inp.split('-')[1])
    dd   = int(inp.split('-')[2])
    
    return datetime.datetime(yyyy,mm,dd).strftime('%A')


def get_is_weekend(input_weekday):
    if input_weekday in ['Saturday', 'Sunday']:
        return 1
    return 0

def get_hour_of_the_day(input_datetime):
    return int(input_datetime.split(' ')[1].split(':')[0])
    
    
bigdf['weekday']    = lmap(pickup_date_to_weekday, bigdf.pickup_date.values)
bigdf['is_weekend'] = lmap(get_is_weekend, bigdf.weekday.values)
bigdf['pickup_hour']= lmap(get_hour_of_the_day, bigdf.pickup_datetime)

bigdf[bigdf.weekday=='Monday'].head(5)

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,Holiday,avg_temp_C,Rain,Fog,Snow,avg_temp_C_scaled,avg_temp_C_anomaly,weekday,is_weekend,pickup_hour
29934,1,2015-01-19 21:32:14,2015-01-19 21:38:41,1,1.10,-73.938889,40.804935,1,N,-73.955513,...,Martin Luther King Jr. Day,4,0.0,0.0,0.0,0.372093,-0.942334,Monday,0,21
29935,2,2015-01-19 10:33:28,2015-01-19 10:37:56,2,0.74,-73.989578,40.756306,1,N,-74.000748,...,Martin Luther King Jr. Day,4,0.0,0.0,0.0,0.372093,-0.942334,Monday,0,10
29936,2,2015-01-19 12:58:06,2015-01-19 13:01:19,5,1.06,-73.957977,40.773483,1,N,-73.955307,...,Martin Luther King Jr. Day,4,0.0,0.0,0.0,0.372093,-0.942334,Monday,0,12
29937,1,2015-01-19 15:47:08,2015-01-19 15:58:15,2,2.40,-73.995155,40.725227,1,N,-73.992134,...,Martin Luther King Jr. Day,4,0.0,0.0,0.0,0.372093,-0.942334,Monday,0,15
29938,2,2015-01-19 12:09:32,2015-01-19 12:13:19,1,0.93,-74.006767,40.744354,1,N,-73.994789,...,Martin Luther King Jr. Day,4,0.0,0.0,0.0,0.372093,-0.942334,Monday,0,12


Sanity check some lengths.  There should be about as many mondays as thursdays, etc.

In [13]:
for weekday in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
    print(weekday, len(bigdf[bigdf.weekday==weekday]))

Monday 153059
Tuesday 165649
Wednesday 174638
Thursday 179529
Friday 180776
Saturday 185746
Sunday 160603


### categorize variables as continuous or discrete

* cts variables useful for regression
* discrete/label to be one-hot-encoded

* first, split bigdf into two X-dataframes for
    1) labels
    2) continuous vars
* get Y-dataframe (vars to be predicted)

In [50]:
# lets give an explicit row index R_ID 
# before we split up the bigdf
# since I don't trust pandas indexes

bigdf['R_ID'] = range(len(bigdf))
print(bigdf.columns)

# lets save this dataframe just in case
print('saving to ',os.getcwd()+'/bigdf.pkl')
bigdf.to_pickle('bigdf.pkl')

Index(['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude', 'RateCodeID',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount', 'pickup_zip',
       'pickup_borough', 'pickup_neighborhood', 'dropoff_zip',
       'dropoff_borough', 'dropoff_neighborhood', 'user_fare', 'tip_fraction',
       'tip_over_twenty_percent', 'pickup_date', 'is_holiday', 'Holiday',
       'avg_temp_C', 'Rain', 'Fog', 'Snow', 'avg_temp_C_scaled',
       'avg_temp_C_anomaly', 'weekday', 'is_weekend', 'pickup_hour', 'R_ID'],
      dtype='object')
saving to  /Users/altoidnerd/Desktop/clo/dotdata/data/bigdf.pkl


In [49]:
os.getcwd()

'/Users/altoidnerd/Desktop/clo/dotdata/data'

In [43]:
# intentionally keep R_ID around in all three
# ... might be handy


label_X_vars = ['VendorID', 
                'passenger_count', 
                'RateCodeID', 
                'store_and_fwd_flag', 
                'payment_type', 
                'extra',
                'mta_tax',
                'improvement_surcharge',
                'pickup_zip', 
                'pickup_borough', 
                'pickup_neighborhood', 
                'dropoff_zip', 
                'dropoff_borough', 
                'dropoff_neighborhood', 
                'is_holiday', 
                'Holiday', 
                'Rain', 
                'Fog', 
                'Snow', 
                'weekday', 
                'is_weekend', 
                'pickup_hour']

# continuous_X_vars = [ col for col in bigdf.columns if col not in label_X_vars ] 
continuous_X_vars = [
#                     'pickup_datetime',    # we will use pickup_hour
#                      'dropoff_datetime',  # we will use pickup_hour
                     'trip_distance',
                     'pickup_longitude',
                     'pickup_latitude',
                     'dropoff_longitude',
                     'dropoff_latitude',
                     'fare_amount',
#                      'extra',        # to labels
#                      'mta_tax',      # to labels
#                      'tip_amount',
                     'tolls_amount',
#                      'improvement_surcharge', # to labels
                     'total_amount',
                     'user_fare',
#                      'tip_fraction',
#                      'tip_over_twenty_percent',
#                      'pickup_date',
#                      'avg_temp_C',
                     'avg_temp_C_scaled',
                     'avg_temp_C_anomaly']

Y_vars = ['tip_fraction', 'tip_over_twenty_percent']

# The Holiday column had some NaN's (Christmas, MLK, NaN if not a holiday)
label_X_df = bigdf[label_X_vars]
label_X_df['Holiday'] = label_X_df['Holiday'].fillna('None')
contonuous_X_df = bigdf[continuous_X_vars]
Y_df = bigdf[Y_vars]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [47]:
print('Continuous Variables')
print('variable','num_vals_taken')
for _ in continuous_X_vars:
    print(_,len(set(bigdf[_])))
    
print('\nDiscrete Variables')
print('variable','num_vals_taken')
for _ in label_X_vars:
    print(_,len(set(bigdf[_])))

Continuous Variables
variable num_vals_taken
trip_distance 2179
pickup_longitude 18930
pickup_latitude 40651
dropoff_longitude 25919
dropoff_latitude 55852
fare_amount 327
tolls_amount 131
total_amount 4300
user_fare 1474
avg_temp_C_scaled 44
avg_temp_C_anomaly 44

Discrete Variables
variable num_vals_taken
VendorID 2
passenger_count 8
RateCodeID 7
store_and_fwd_flag 2
payment_type 4
extra 10
mta_tax 2
improvement_surcharge 2
pickup_zip 163
pickup_borough 5
pickup_neighborhood 40
dropoff_zip 177
dropoff_borough 5
dropoff_neighborhood 42
is_holiday 2
Holiday 15
Rain 2
Fog 2
Snow 2
weekday 7
is_weekend 2
pickup_hour 24


In [36]:
continuous_X_vars

['pickup_datetime',
 'dropoff_datetime',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'user_fare',
 'tip_fraction',
 'tip_over_twenty_percent',
 'pickup_date',
 'avg_temp_C',
 'avg_temp_C_scaled',
 'avg_temp_C_anomaly',
 'R_ID']

In [23]:
label_X_df.head()

,VendorID,passenger_count,RateCodeID,store_and_fwd_flag,payment_type,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,is_holiday,Holiday,Rain,Fog,Snow,weekday,is_weekend,pickup_hour
0,2,1,1,N,2,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,0,None,0.0,0.0,1.0,Tuesday,0,11
1,1,1,1,Y,2,10065,Manhattan,Upper East Side,10025,Manhattan,Upper West Side,0,None,0.0,0.0,1.0,Tuesday,0,14
2,1,1,1,N,2,10011,Manhattan,Chelsea and Clinton,10018,Manhattan,Chelsea and Clinton,0,None,0.0,0.0,1.0,Tuesday,0,17
3,1,1,1,N,1,10128,Manhattan,Upper East Side,10021,Manhattan,Upper East Side,0,None,0.0,0.0,1.0,Tuesday,0,8
4,1,2,1,N,1,10128,Manhattan,Upper East Side,10023,Manhattan,Upper West Side,0,None,0.0,0.0,1.0,Tuesday,0,15


In [19]:
contonuous_X_df.head()

,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,extra,mta_tax,...,improvement_surcharge,total_amount,user_fare,tip_fraction,tip_over_twenty_percent,pickup_date,avg_temp_C,avg_temp_C_scaled,avg_temp_C_anomaly,R_ID
0,2015-01-06 11:39:29,2015-01-06 11:49:15,1.78,-73.999619,40.743599,-73.992203,40.764019,9.0,0.0,0.5,...,0.3,9.80,9.3,0.000000,0,2015-01-06,-6,0.139535,-1.890643,0
1,2015-01-06 14:01:03,2015-01-06 14:12:00,1.80,-73.968254,40.767300,-73.967339,40.788025,9.0,0.0,0.5,...,0.3,9.80,9.3,0.000000,0,2015-01-06,-6,0.139535,-1.890643,1
2,2015-01-06 17:35:31,2015-01-06 17:43:34,0.90,-73.998901,40.745388,-73.990288,40.751408,7.0,1.0,0.5,...,0.3,8.80,8.3,0.000000,0,2015-01-06,-6,0.139535,-1.890643,2
3,2015-01-06 08:22:22,2015-01-06 08:31:59,0.70,-73.945030,40.779266,-73.951805,40.769947,7.5,0.0,0.5,...,0.3,9.95,7.8,0.211538,1,2015-01-06,-6,0.139535,-1.890643,3
4,2015-01-06 15:17:45,2015-01-06 15:40:41,2.90,-73.955978,40.787697,-73.983749,40.774132,16.0,0.0,0.5,...,0.3,18.55,16.3,0.107362,0,2015-01-06,-6,0.139535,-1.890643,4


In [20]:
Y_df.head()

,tip_fraction,tip_over_twenty_percent,R_ID
0,0.000000,0,0
1,0.000000,0,1
2,0.000000,0,2
3,0.211538,1,3
4,0.107362,0,4


In [33]:
label_X_df.head()

,VendorID,passenger_count,RateCodeID,store_and_fwd_flag,payment_type,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,is_holiday,Holiday,Rain,Fog,Snow,weekday,is_weekend,pickup_hour
0,2,1,1,N,2,10011,Manhattan,Chelsea and Clinton,10036,Manhattan,Chelsea and Clinton,0,None,0.0,0.0,1.0,Tuesday,0,11
1,1,1,1,Y,2,10065,Manhattan,Upper East Side,10025,Manhattan,Upper West Side,0,None,0.0,0.0,1.0,Tuesday,0,14
2,1,1,1,N,2,10011,Manhattan,Chelsea and Clinton,10018,Manhattan,Chelsea and Clinton,0,None,0.0,0.0,1.0,Tuesday,0,17
3,1,1,1,N,1,10128,Manhattan,Upper East Side,10021,Manhattan,Upper East Side,0,None,0.0,0.0,1.0,Tuesday,0,8
4,1,2,1,N,1,10128,Manhattan,Upper East Side,10023,Manhattan,Upper West Side,0,None,0.0,0.0,1.0,Tuesday,0,15


In [26]:
 testcols=['VendorID', 'passenger_count', 'RateCodeID', 'store_and_fwd_flag',
        'payment_type', 'pickup_zip', 'pickup_borough', 'pickup_neighborhood',
        'dropoff_zip', 'dropoff_borough', 'dropoff_neighborhood',
        'Holiday', 'weekday']

In [34]:
le = preprocessing.LabelEncoder()



# testdf = label_X_df[testcols]
# test2df = testdf.apply(le.fit_transform)
# test2df.head()

label_X2_df = label_X_df.apply(le.fit_transform)
label_X2_df.head()

,VendorID,passenger_count,RateCodeID,store_and_fwd_flag,payment_type,pickup_zip,pickup_borough,pickup_neighborhood,dropoff_zip,dropoff_borough,dropoff_neighborhood,is_holiday,Holiday,Rain,Fog,Snow,weekday,is_weekend,pickup_hour
0,1,1,0,0,1,9,2,8,33,2,8,0,11,0,0,1,5,0,11
1,0,1,0,1,1,39,2,36,22,2,39,0,11,0,0,1,5,0,14
2,0,1,0,0,1,9,2,8,15,2,8,0,11,0,0,1,5,0,17
3,0,1,0,0,0,41,2,36,18,2,38,0,11,0,0,1,5,0,8
4,0,2,0,0,0,41,2,36,20,2,39,0,11,0,0,1,5,0,15


In [30]:
onehot = preprocessing.OneHotEncoder()

onehot.fit(test2df)

onehotlabels = onehot.transform(test2df).toarray()

In [32]:
onehotlabels.shape

(1200000, 477)

In [35]:
contonuous_X_df.apply()

,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,extra,mta_tax,...,improvement_surcharge,total_amount,user_fare,tip_fraction,tip_over_twenty_percent,pickup_date,avg_temp_C,avg_temp_C_scaled,avg_temp_C_anomaly,R_ID
0,2015-01-06 11:39:29,2015-01-06 11:49:15,1.78,-73.999619,40.743599,-73.992203,40.764019,9.0,0.0,0.5,...,0.3,9.80,9.30,0.000000,0,2015-01-06,-6,0.139535,-1.890643,0
1,2015-01-06 14:01:03,2015-01-06 14:12:00,1.80,-73.968254,40.767300,-73.967339,40.788025,9.0,0.0,0.5,...,0.3,9.80,9.30,0.000000,0,2015-01-06,-6,0.139535,-1.890643,1
2,2015-01-06 17:35:31,2015-01-06 17:43:34,0.90,-73.998901,40.745388,-73.990288,40.751408,7.0,1.0,0.5,...,0.3,8.80,8.30,0.000000,0,2015-01-06,-6,0.139535,-1.890643,2
3,2015-01-06 08:22:22,2015-01-06 08:31:59,0.70,-73.945030,40.779266,-73.951805,40.769947,7.5,0.0,0.5,...,0.3,9.95,7.80,0.211538,1,2015-01-06,-6,0.139535,-1.890643,3
4,2015-01-06 15:17:45,2015-01-06 15:40:41,2.90,-73.955978,40.787697,-73.983749,40.774132,16.0,0.0,0.5,...,0.3,18.55,16.30,0.107362,0,2015-01-06,-6,0.139535,-1.890643,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,2015-12-25 23:10:23,2015-12-25 23:18:09,1.90,-73.979012,40.785275,-73.982971,40.761028,8.5,0.5,0.5,...,0.3,12.25,9.30,0.263441,1,2015-12-25,17,0.674419,0.290468,1199995
1199996,2015-12-25 09:48:10,2015-12-25 09:56:42,1.81,-73.955025,40.769787,-73.978012,40.762974,8.0,0.0,0.5,...,0.3,10.56,8.30,0.212048,1,2015-12-25,17,0.674419,0.290468,1199996
1199997,2015-12-25 14:39:38,2015-12-25 15:01:38,9.50,-73.873070,40.774132,-73.979591,40.781937,28.0,0.0,0.5,...,0.3,41.21,33.84,0.203014,1,2015-12-25,17,0.674419,0.290468,1199997
1199998,2015-12-25 16:05:09,2015-12-25 16:09:12,1.10,-73.988861,40.773930,-73.999367,40.761189,5.5,0.0,0.5,...,0.3,7.56,5.80,0.217241,1,2015-12-25,17,0.674419,0.290468,1199998


In [1]:
import sys
import ast
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector
from sqlalchemy import create_engine
from operator import add, truediv
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression
from scipy.stats.stats import pearsonr
from collections import OrderedDict

port = 3306
username = "rhombus"
password = "Rhombus_2019z"
end_point = "afwic.c9fkygyhkkab.us-gov-west-1.rds.amazonaws.com"
afwic = mysql.connector.connect(user=username, password=password, host=end_point, port=port)
afwiccon = create_engine('mysql+mysqldb://{}:{}@{}:{}'.format(username,password,end_point,port))

username = "rhombus"
password = "rhombuspower"
end_point = "quantum.c9fkygyhkkab.us-gov-west-1.rds.amazonaws.com"
quantum = mysql.connector.connect(user=username, password=password, host=end_point, port=port)
quantumcon = create_engine('mysql+mysqldb://{}:{}@{}:{}'.format(username,password,end_point,port))

df_availability = pd.read_sql(''' SELECT * FROM LIMS_EV.AFKCA_MCH_PSH_AVAILABILITY ''',afwic)
df_availability2= pd.read_sql(''' SELECT * FROM LIMS_EV.AFKCA_MCH_PSH_AVAILABILITY_AVG''',afwic)
df_budget = pd.read_sql(''' SELECT * FROM PROGRAMMING_UI.DT_ABIDES_AFKCA ''',quantum)
df_afkca = pd.read_sql(''' SELECT * FROM PLANNING_UI_DEPLOY.LOOKUP_CONNECTION_AFKCA ''',quantum)
saveeq = {'A010A':'A-10','A010C':'A-10',
       'AC130H':'AC-130H/U','AC130J':'AC-130J','AC130U':'AC-130H/U','AC130W':'AC-130W',
       'B001B':'B-1','B002A':'B-2',
       'B052G':'B-52','B052H':'B-52','B052C':'B-52','B052D':'B-52','B052E':'B-52','B052F':'B-52',
       'C012A':'C-12','C012C':'C-12','C012D':'C-12','C012F':'C-12','C012J':'C-12',
       'C017A':'C-17',
       'C020A':'C-20','C020B':'C-20','C020C':'C-20','C020E':'C-20','C020H':'C-20','C020K':'C-20',
       'C021A':'C-21',
       'C032A':'C-32','C032B':'C-32',
       'C037A':'C-37','C037B':'C-37',
       'C040A':'C-40','C040B':'C-40','C040C':'C-40',
       'C130H':'C-130H','C130J':'C-130J',
       'CV022B':'CV-22',
       'E003A':'E-3','E003B':'E-3','E003C':'E-3','E003G':'E-3',
       'E004B':'E-4','E008A':'E-8','E008C':'E-8',
       'EC130E':'EC-130 CCALL','EC130H':'EC-130 CCALL','EC130J':'EC-130 CCALL',
       'F015C':'F-15CD','F015D':'F-15CD','F015E':'F-15E',
       'F016C':'F-16CD','F016D':'F-16CD',
       'F022A':'F-22',
       'F035A':'F-35A',
       'F117A':'F-117',
       'HC130J':'HC-130J',
       'HC130N':'HC-130N/P',
       'HC130P':'HC-130N/P',
       'HH060A':'HH-60','HH060G':'HH-60','HH060U':'HH-60',
       'KC010A':'KC-10',
       'KC135A':'KC-135','KC135D':'KC-135','KC135E':'KC-135','KC135R':'KC-135','KC135T':'KC-135','KC135Q':'KC-135',
       'LC130H':'C-130H',
       'MC012W':'MC-12',
       'MC130H':'MC-130H','MC130J':'MC-130J',
       'MQ001B':'MQ-1','MQ009A':'MQ-9',
       'OC135B':'ARMS CONTROL (OC-135B)',
       'RC026B':'RC-26',
       'RQ004A':'RQ-4','RQ004B':'RQ-4',
       'T001A':'T-1',
       'T006A':'T-6',
       'T038A':'T-38','T038C':'T-38',
       'TH001H':'TH-1','TH001F':'TH-1',
       'U002S':'U-2','TU002R':'U-2','U002R':'U-2',
       'UH001H':'UH-1N NDO','UH001N':'UH-1N NDO','UH001V':'UH-1N NDO','UH001F':'UH-1N NDO','UH001P':'UH-1N NDO',
       'VC025A':'VC-25A',
       'WC130J':'WC-130J',
       'WC135B':'WC-135','WC135C':'WC-135','WC135W':'WC-135'}
lut = pd.DataFrame.from_dict(saveeq,orient='index')
lut.columns = ['AFKCA']
lut.index.names = ['EQUIPMENT_DESIGNATOR']
lut.head()




afkca_type_dict = {'BOMBERS': ['B-1', 'B-2', 'B-52'],
 'C2ISR': ['ARMS CONTROL (OC-135B)', 'E-3', 'E-4', 'E-8', 'MC-12', 'MQ-1', 'MQ-9', 'RC-26', 'RQ-4', 'U-2', 'WC-130J'],
 'CSAR': ['HC-130J', 'HC-130N/P'],
 'EW ASSETS': ['EC-130 CCALL'],
 'FIGHTERS': ['A-10', 'F-117', 'F-15CD', 'F-15E', 'F-16CD', 'F-22', 'F-35A'],
 'OSA/EA': ['C-12', 'C-20', 'C-21', 'C-32', 'C-37', 'C-40', 'VC-25A'],
 'ROTARY WING': ['HH-60', 'UH-1N NDO'],
 'SOF': ['CV-22', 'MC-130J'],
 'STRATLIFT': ['C-17'],
 'TACLIFT': ['C-130H', 'C-130J'],
 'TANKERS': ['KC-10', 'KC-135'],
 'TRAINERS': ['T-1', 'T-38', 'T-6', 'TH-1']}

# ALL_ACS = pd.read_sql('''SELECT * FROM PROGRAMMING_UI.DT_ABIDES_AFKCA where OPS = 'ACS';''', quantum)
# ALL_SSS = pd.read_sql('''SELECT * FROM PROGRAMMING_UI.DT_ABIDES_AFKCA where OPS = 'SSS';''', quantum)


# def extract_afkca(x):
#     x = x.replace('{','[').replace('}',']')
#     x = '{{{}}}'.format(x[1:-1])
#     x = ast.literal_eval(x)['AFKCA']
#     return x

def extract_afkca(x):
    if x is None:
        return []
    else:
        x = x.replace('{','[').replace('}',']')
        x = '{{{}}}'.format(x[1:-1])
        x = ast.literal_eval(x)['AFKCA']
        return x


def extract_budget(afkca='F-15CD', kind='MILPERS', retlist=None):
    def lmap(func, alist):
        return list(map(func, alist))

    if retlist is None:
        return np.array(lmap(float, df_budget[df_budget['AFKCA'] == afkca].iloc[0][kind].split(',')))

    elif retlist>0:
        return lmap(float, df_budget[df_budget['AFKCA'] == afkca].iloc[0][kind].split(','))


def get_min_max(seq):
    min_ = min(seq)
    max_ = max(seq)
    seq = (seq-min_)/(max_ - min_)
    return [(min_,max_),seq]


def get_deltas(seq):
    deltas = [] 
    i=0
    while i < len(seq)-1:
        deltas.append(seq[i+1]-seq[i])
        i+=1
    return np.array(deltas)



def get_availability(afkca='F-15CD', start_year=2001, end_year=2018):
    return np.array(list(df_availability[df_availability.AFKCA==afkca].AVAILABILITY)[start_year-2000:end_year-1999])



def get_scatter_df(afkca_='F-15CD',suppress_cols=[],use_avg=False):

    # availability df adf
    
    adf=df_availability[df_availability.AFKCA==afkca_]#[['FISCAL_YEAR','AVAILABILITY']]
    adf=adf[adf.FISCAL_YEAR > 2000]
    
    if use_avg:
        adf=df_availability2[df_availability2.AFKCA==afkca_]#[['FISCAL_YEAR','AVAILABILITY']]
        adf=adf[adf.FISCAL_YEAR > 2000]
    
    # budget df bdf
    b=OrderedDict()
    b['FY']                    = range(2001,2019)
    b['OM']                    = extract_budget(afkca=afkca_,kind='OM')[:-5]
   
    b['MILPERS']               = extract_budget(afkca=afkca_,kind='MILPERS')[:-5]
    b['AFKCA_TAI']             = extract_budget(afkca=afkca_, kind='FORCES_TAI')[:-5]
    
    bdf=pd.DataFrame.from_dict(b)
    
    df=bdf[bdf.FY>=min(adf.FISCAL_YEAR)]
    df['AVAIL'] = list(adf.AVAILABILITY)

    df=df[df.AFKCA_TAI>0]
    df['OM/TAI']                = np.array(df['OM'])/np.array(df['AFKCA_TAI'])
    df['MILPERS/TAI']           = np.array(df['MILPERS'])/np.array(df['AFKCA_TAI'])
    
    df = df[['FY', 'OM', 'MILPERS', 'AFKCA_TAI','OM/TAI', 'MILPERS/TAI','AVAIL']]
    df = df.drop(columns=suppress_cols)

    return df #.drop(columns=suppress_cols)


afk='F-22'
print(afk)

pd.plotting.scatter_matrix(get_scatter_df(afk,suppress_cols=['AFKCA_TAI', 'OM/TAI', 'MILPERS/TAI']))#'OM','MILPERS']))#, 'AFKCA_TAI']))
plt.show()

plt.hist(list(get_scatter_df(afk).FY),bins=range(2001,2019))
plt.show()

def get_connected_df(afkca_='F-15CD', use_avg=False, ret='all'):
    

    # availability df adf
    adf=df_availability[df_availability.AFKCA==afkca_]
    adf=adf[adf.FISCAL_YEAR > 2000]

    if use_avg:
        adf=df_availability2[df_availability2.AFKCA==afkca_]
        adf=adf[adf.FISCAL_YEAR > 2000]

    # budget df bdf
    b=OrderedDict()
    b['FY']                    = range(2001,2019)
    b['AFKCA_TAI']             = extract_budget(afkca=afkca_, kind='FORCES_TAI')[:-5]
    b['OM']                    = extract_budget(afkca=afkca_,kind='OM')[:-5]
    b['MILPERS']               = extract_budget(afkca=afkca_,kind='MILPERS')[:-5]
    b['AFKCA_TAI']             = extract_budget(afkca=afkca_, kind='FORCES_TAI')[:-5]
    bdf=pd.DataFrame.from_dict(b)


    ACS_connections = extract_afkca(df_afkca[df_afkca.AFKCA == afkca_].CONNECTED_AFKCA_ACS.iloc[0]) 
    ACS_BUDGETS = OrderedDict()
    for afk in ACS_connections:
        milpers = extract_budget(afkca=afk, kind='MILPERS')
        om      = extract_budget(afkca=afk, kind='OM')
        rdte    = extract_budget(afkca=afk, kind='RDTE')
        proc    = extract_budget(afkca=afk, kind='PROCUREMENT')
        milcon  = extract_budget(afkca=afk, kind='MILCON')
        other   = extract_budget(afkca=afk, kind='OTHER')
    #       ACS_BUDGETS[afk] = 1/TOTAL_TAI*(milpers + om + rdte + proc + milcon + other)
        ACS_BUDGETS[afk] =             (milpers + om + rdte + proc + milcon + other)[:-5]
    ACS_BUDGETS['FY']    = range(2001,2019)
    ACS_df = pd.DataFrame.from_dict(ACS_BUDGETS)

    SSS_connections = extract_afkca(df_afkca[df_afkca.AFKCA == afkca_].CONNECTED_AFKCA_SSS.iloc[0])
    SSS_BUDGETS = OrderedDict()
    for afk in SSS_connections:
        milpers = extract_budget(afkca=afk, kind='MILPERS')
        om      = extract_budget(afkca=afk, kind='OM')
        rdte    = extract_budget(afkca=afk, kind='RDTE')
        proc    = extract_budget(afkca=afk, kind='PROCUREMENT')
        milcon  = extract_budget(afkca=afk, kind='MILCON')
        other   = extract_budget(afkca=afk, kind='OTHER')
    #       SSS_BUDGETS[afk] = 1/TOTAL_TAI*(milpers + om + rdte + proc + milcon + other)
        SSS_BUDGETS[afk] = (milpers + om + rdte + proc + milcon + other)[:-5]
    SSS_BUDGETS['FY']    = range(2001,2019)
    SSS_df = pd.DataFrame.from_dict(SSS_BUDGETS)


    bdf=bdf[bdf.FY>=min(adf.FISCAL_YEAR)]
    ACS_df=ACS_df[ACS_df.FY>=min(adf.FISCAL_YEAR)]
    SSS_df=SSS_df[SSS_df.FY>=min(adf.FISCAL_YEAR)]

    df=pd.DataFrame()
    df['FY']      = adf.FISCAL_YEAR
    df['AVAIL']   = adf.AVAILABILITY
    # df['OM']      = bdf.OM
    # df['MILPERS'] = bdf.MILPERS

    df = df.merge(bdf,    on='FY')
    df = df.merge(ACS_df, on='FY')
    df = df.merge(SSS_df, on='FY')

    if ret=='all':
        return df
    elif ret=='adf':
        return adf
    elif ret=='bdf':
        return bdf
    elif ret=='ACS':
        return ACS_df
    elif ret=='SSS':
        return SSS_df
    
    

def get_all_ACS_SSS_df(afkca_='F-15CD', use_avg=True, ret='all'):
    

    # availability df adf

    if use_avg:
        adf=df_availability2[df_availability2.AFKCA==afkca_]
        adf=adf[adf.FISCAL_YEAR > 2000]
    else:
        adf=df_availability[df_availability.AFKCA==afkca_]
        adf=adf[adf.FISCAL_YEAR > 2000]

    # budget df bdf
    b=OrderedDict()
    b['FY']                    = range(2001,2019)
    b['AFKCA_TAI']             = extract_budget(afkca=afkca_, kind='FORCES_TAI')[:-5]
    
    b['OM']                    = extract_budget(afkca=afkca_, kind='OM')[:-5]
    b['OM_CIVPERS']            = extract_budget(afkca=afkca_, kind='OM_CIVPERS')[:-5]
    b['OM_FHP']                = extract_budget(afkca=afkca_, kind='OM_FHP')[:-5]
    b['OM_FUEL']               = extract_budget(afkca=afkca_, kind='OM_FUEL')[:-5]
    b['OM_WSS']                = extract_budget(afkca=afkca_, kind='OM_WSS')[:-5]
    b['OM_REMAINING']          = extract_budget(afkca=afkca_, kind='OM_REMAINING')[:-5]
    
    b['MILPERS']               = extract_budget(afkca=afkca_, kind='MILPERS')[:-5]
    b['MILPERS_ENLISTED']      = extract_budget(afkca=afkca_, kind='MILPERS_ENLISTED')[:-5]
    b['MILPERS_OFFICER']       = extract_budget(afkca=afkca_, kind='MILPERS_OFFICER')[:-5]
    b['MILPERS_REMAINING']     = extract_budget(afkca=afkca_, kind='MILPERS_REMAINING')[:-5]
    
    bdf=pd.DataFrame.from_dict(b)


    #ACS_connections = extract_afkca(df_afkca[df_afkca.AFKCA == afkca_].CONNECTED_AFKCA_ACS.iloc[0])
    # 48 of these
    ACS_afkcas = sorted(list(df_budget[df_budget.OPS=='ACS'].AFKCA))
    ACS_BUDGETS = OrderedDict()
    for afk in ACS_afkcas:
        milpers = extract_budget(afkca=afk, kind='MILPERS')
        om      = extract_budget(afkca=afk, kind='OM')
        rdte    = extract_budget(afkca=afk, kind='RDTE')
        proc    = extract_budget(afkca=afk, kind='PROCUREMENT')
        milcon  = extract_budget(afkca=afk, kind='MILCON')
        other   = extract_budget(afkca=afk, kind='OTHER')
    #       ACS_BUDGETS[afk] = 1/TOTAL_TAI*(milpers + om + rdte + proc + milcon + other)
        ACS_BUDGETS[afk] =             (milpers + om + rdte + proc + milcon + other)[:-5]
    ACS_BUDGETS['FY']    = range(2001,2019)
    ACS_df = pd.DataFrame.from_dict(ACS_BUDGETS)

    #SSS_connections = extract_afkca(df_afkca[df_afkca.AFKCA == afkca_].CONNECTED_AFKCA_SSS.iloc[0])
    SSS_afkcas = sorted(list(df_budget[df_budget.OPS=='SSS'].AFKCA))
    SSS_BUDGETS = OrderedDict()
    for afk in SSS_afkcas:
        milpers = extract_budget(afkca=afk, kind='MILPERS')
        om      = extract_budget(afkca=afk, kind='OM')
        rdte    = extract_budget(afkca=afk, kind='RDTE')
        proc    = extract_budget(afkca=afk, kind='PROCUREMENT')
        milcon  = extract_budget(afkca=afk, kind='MILCON')
        other   = extract_budget(afkca=afk, kind='OTHER')
    #       SSS_BUDGETS[afk] = 1/TOTAL_TAI*(milpers + om + rdte + proc + milcon + other)
        SSS_BUDGETS[afk] = (milpers + om + rdte + proc + milcon + other)[:-5]
    SSS_BUDGETS['FY']    = range(2001,2019)
    SSS_df = pd.DataFrame.from_dict(SSS_BUDGETS)


    bdf=bdf[bdf.FY>=min(adf.FISCAL_YEAR)]
    ACS_df=ACS_df[ACS_df.FY>=min(adf.FISCAL_YEAR)]
    SSS_df=SSS_df[SSS_df.FY>=min(adf.FISCAL_YEAR)]

    df=pd.DataFrame()
    df['FY']      = adf.FISCAL_YEAR
    df['AVAIL']   = adf.AVAILABILITY
    # df['OM']      = bdf.OM
    # df['MILPERS'] = bdf.MILPERS

    df = df.merge(bdf,    on='FY')
    df = df.merge(ACS_df, on='FY')
    df = df.merge(SSS_df, on='FY')

    if ret=='all':
        return df
    elif ret=='adf':
        return adf
    elif ret=='bdf':
        return bdf
    elif ret=='ACS':
        return ACS_df
    elif ret=='SSS':
        return SSS_df
    

# aircraft = sorted(list(set(lut.AFKCA)))
# aircraft.remove('ARMS CONTROL (OC-135B)')
    
# for afk in aircraft:
#     print(afk)
#     try:
#         df=get_all_ACS_SSS_df(afk).corr()
#     except:
#         e = sys.exc_info()[0]
#         print(e)
    
# df

# df = get_all_ACS_SSS_df('F-15CD')
# df
    




F-22
